In [1]:
import os

import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from scipy.stats import pearsonr
from tqdm import tqdm

from data import SGDDataset
from metrics import ConversationalEditDistance, StructuralEditDistance

In [2]:
embeddings_dir = 'conversation-similarity/cache'

os.makedirs(embeddings_dir, exist_ok=True)

In [3]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

model_name = 'all-MiniLM-L12-v2'
model = SentenceTransformer(model_name, device=device)

In [4]:
dataset = SGDDataset('dstc8-schema-guided-dialogue')

In [5]:
target_services = {'Events_1', 'Events_2', 'Events_3'}
target_dialog_ids = []
for idx, dialog in enumerate(dataset.dialogs):
    if len(set(dialog.services).union(target_services)) <= 3:
        target_dialog_ids.append(idx)

In [6]:
conved = ConversationalEditDistance(
    is_inverted=False, normalize=False, substitution_weight=2.2,
)
structed = StructuralEditDistance(
    is_inverted=False, normalize=True, substitution_weight=2.0,
)

In [7]:
np.random.seed(1234)
dialog_ids = np.random.choice(target_dialog_ids, 200, replace=False)

In [8]:
for dialog_id in tqdm(dialog_ids):
    dataset.dialogs[dialog_id].compute_embeddings(
        embeddings_dir, model, model_name,
    )

100%|██████████| 200/200 [00:11<00:00, 17.37it/s]


In [9]:
conved_distances, structed_distances = [], []
for i in tqdm(dialog_ids):
    for j in dialog_ids:
        if i < j:
            conved_distances.append(conved(dataset[i], dataset[j]))
            structed_distances.append(structed(dataset[i], dataset[j]))

100%|██████████| 200/200 [01:17<00:00,  2.60it/s]


In [10]:
pearsonr(structed_distances, conved_distances)

PearsonRResult(statistic=-0.12500056493455003, pvalue=4.0569453640095733e-70)